In [ ]:
# 匯入套件
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")              #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(options = options)

# 放置 FB 個人發文的基本資訊
listPost = []

'''
以下定義了一些方法，作為爬蟲流程
'''

# 前往 FB
def begin():    
    # 走訪網址
    driver.get('https://www.facebook.com/')

# 登入
def login():
    # 輸入帳號
    inputEmail = driver.find_element_by_id('email') 
    inputEmail.send_keys("你的帳號")

    # 輸入密碼
    inputPwd = driver.find_element_by_id('pass')
    inputPwd.send_keys("你的密碼")

    # 按下登入/送出鈕
    btnSubmit = driver.find_element_by_css_selector('button[type="submit"][name="login"]')
    btnSubmit.click()
    
    # 強制等待
    sleep(3)

# 走訪頁面
def visit():
    driver.get('https://www.facebook.com/JesseTang11')

# 滾動頁面
def scroll():
    '''
    totalHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 計無效滾動次數
    limit => 最大無效滾動次數
    wait_second => 每次滾動後的強制等待時間
    '''
    totalHeight = 0
    offset = 0
    count = 0
    limit = 3
    wait_second = 3
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏
        offset += 600
        '''
        
        # 捲動的 js code
        js_code = f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        '''
        
        # 執行 js code
        driver.execute_script(js_code)
        
        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(wait_second)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        totalHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        );
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset >= totalHeight:
            count += 1
            
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 600:
            break

# 分析元素內容
def parse():
    # 確認當前動個人態牆的發文數量
    cssSelectorPost = 'div[data-ad-comet-preview="message"] span[dir="auto"]'

    try:
        # 取得元素
        div = driver.find_elements_by_css_selector(cssSelectorPost)

        # 將每個發文的內容擷取出來
        for index, elm in enumerate(div):
            print("index: {}".format(index))
            print("text: {}".format(elm.text))
            print()
            
            # 將資料新增到 list 當中
            listPost.append({
                "index": index,
                "text":elm.text
            })
        
        # 將放置發文的 list，以 JSON 格式存入檔案
        fp = open("FB.json", "w",encoding='UTF-8')
        fp.write( json.dumps(listPost, ensure_ascii=False) )
        fp.close()

    except TimeoutException:
        print('等待逾時！')

# 關閉瀏覽器
def close():
    # 關閉瀏覽器
    driver.quit()

# 主程式
if __name__ == '__main__':
    begin()
    login()
    visit()
    scroll()
    parse()
    close()